In [ ]:
import moviepy.editor
import os
import random
import fnmatch
import math

base_directory = r'D:\2022-불법영상물-유튜브'

ext = "*mp4"
length = 180 # 3분 (180초)
short_edge = 256

labels = ["agree", "nonagree"]

for label in labels:
    directory = os.path.join(base_directory, label)
    dst_directory = os.path.join(base_directory + "-after", label)

    if not os.path.exists(dst_directory):
        os.makedirs(dst_directory)

    # compile list of videos
    inputs = [os.path.join(directory,f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and fnmatch.fnmatch(f, ext)]

    for idx, name in enumerate(inputs):
        orig_file_name = os.path.basename(name)

        # import to moviepy
        clip = moviepy.editor.VideoFileClip(name)

        # 강제로 256x256으로 만듬. 비율 달라지는건 상관 x
        clip = clip.resize((short_edge, short_edge))

        clip_start = []

        for i in range (0, math.floor(clip.duration)):
            if i % 30 == 0 and i + 30 < clip.duration:
                clip_start.append(i)
        #print(clip_start)

        if (len(clip_start) > 5):
            clip_start = random.sample(clip_start, 5)

        clip_start.sort()

        with open(os.path.join(base_directory + "-after", label, orig_file_name.split("_")[0] + ".txt"), 'w') as f:
            for c in clip_start:
                f.write(str(c) + "\n")

        clips = []

        for start in clip_start:
            # cut a subclip
            clips.append(clip.subclip(start,start+length))

        for idx, c in enumerate(clips):
            c.write_videofile(os.path.join(base_directory + "-after", label, orig_file_name.split("_")[0] + "-" + f'{idx+1:03d}.mp4'))